Copyright **`(c)`** 2023 Giovanni Squillero `<giovanni.squillero@polito.it>`  
[`https://github.com/squillero/computational-intelligence`](https://github.com/squillero/computational-intelligence)  
Free for personal or classroom use; see [`LICENSE.md`](https://github.com/squillero/computational-intelligence/blob/master/LICENSE.md) for details.  

In [95]:
from itertools import product
from random import random, randint, shuffle, seed, choice
import numpy as np
from scipy import sparse
from functools import reduce
from copy import copy
import math

In [44]:
def make_set_covering_problem(num_points, num_sets, density):
    """Returns a sparse array where rows are sets and columns are the covered items"""
    seed(num_points*2654435761+num_sets+density)
    sets = sparse.lil_array((num_sets, num_points), dtype=bool)
    for s, p in product(range(num_sets), range(num_points)):
        if random() < density:
            sets[s, p] = True
    for p in range(num_points):
        sets[randint(0, num_sets-1), p] = True
    return sets

# Halloween Challenge

Find the best solution with the fewest calls to the fitness functions for:

* `num_points = [100, 1_000, 5_000]`
* `num_sets = num_points`
* `density = [.3, .7]` 

In [45]:
NUM_POINTS = 100
NUM_SETS = NUM_POINTS
DENSITY = 0.3
NUM_STEPS = 10_000


In [46]:
SETS = make_set_covering_problem(NUM_POINTS, NUM_SETS, DENSITY)
#print("Element at row=42 and column=42:", SETS[42, 42])

# Hill climbing



In [113]:
def fitness1(state):
    cost = sum(state)
    valid = np.all(
        reduce(
            np.logical_or,
            [SETS.toarray()[i] for i, t in enumerate(state) if t],
            np.array([False for _ in range(NUM_POINTS)]),
        )
    )
    return valid, -cost

def fitness2(state):
    cost = sum(state)
    valid = np.sum(
        reduce(
            np.logical_or,
            [SETS.toarray()[i] for i, t in enumerate(state) if t],
            np.array([False for _ in range(NUM_POINTS)]),
        )
    )
    return valid, -cost

def fitness3(state):
    cost = sum(state)
    valid = np.sum(
        reduce(
            np.logical_or,
            [SETS.toarray()[i] for i, t in enumerate(state) if t],
            np.array([False for _ in range(NUM_POINTS)]),
        )
    )
    return valid, cost

fitness = fitness2

In [48]:
def tweak(state):
    new_state = copy(state)
    index = randint(0, NUM_SETS - 1)
    new_state[index] = not new_state[index]
    return new_state

In [ ]:
def local_search(current_state):
    # Implementa la tua ricerca locale qui
    # Questa dovrebbe migliorare la soluzione fornita
    for step in range(NUM_STEPS):
        new_state = tweak(current_state)
        if fitness(new_state) >= fitness(current_state):
            current_state = new_state
            print(fitness(current_state))
    return current_state

current_state = [False for _ in range(NUM_SETS)]
print(fitness(local_search(current_state)))

# Simulated annealing



In [ ]:
current_state = [False for _ in range(NUM_SETS)]
print(fitness(current_state))

for step in range(NUM_STEPS):
    new_state = tweak(current_state)
    x, y = fitness(current_state)
    x1, y1, = fitness(new_state)

    probability =  1 / math.exp( ( (x-y) - (x1-y1) ) / ( (NUM_STEPS + 1) - step) )
    print ("===>", probability, (NUM_STEPS + 1) - step )

    if (probability >= random() or fitness(new_state) >= fitness(current_state) ):
        current_state = new_state
        print(fitness(current_state))

In [ ]:
#temp compresa fra 0 e 1

current_state = [False for _ in range(NUM_SETS)]
print(fitness(current_state))

for step in range(NUM_STEPS):
    new_state = tweak(current_state)
    x, y = fitness(current_state)
    x1, y1, = fitness(new_state)

    temp = (( (NUM_STEPS + 1) - step)  / NUM_STEPS )
    #probability =  1 / math.exp( ( (x-y) - (x1-y1) ) /  temp
    probability = temp

    print ("===>", probability, (NUM_STEPS + 1) - step )

    if (probability >= random() or fitness(new_state) >= fitness(current_state) ):
        current_state = new_state
        print(fitness(current_state))

# Tabu 

In [105]:
NUM_NEIGHBORS = 20
TABU_LIST_MAX_SIZE = 5

In [106]:
def is_valid(state):
    a, b = fitness(state)
    if a == NUM_POINTS:
        return True
    return False

In [ ]:
tabu_list = []
current_state = [choice([True, False]) for _ in range(NUM_SETS)]

for iteration in range(NUM_STEPS):
    neighbors = [tweak(current_state) for _ in range(NUM_NEIGHBORS)]
    
    # Seleziona candidati validi non presenti nella lista tabù
    candidates = [sol for sol in neighbors if is_valid(sol) and sol not in tabu_list]
    
    if not candidates:
        # Nessun candidato valido disponibile, potrebbe essere un punto di arresto
        break
    
    # Valuta candidati e scegli il migliore
    best_candidate = min(candidates, key=fitness3)
    
    # Aggiorna la lista tabù con la mossa corrente
    tabu_list.append(best_candidate)
    
    # Rimuovi le mosse più vecchie dalla lista tabù se è troppo lunga
    if len(tabu_list) > TABU_LIST_MAX_SIZE:
        tabu_list.pop(0)
    
    # Aggiorna la soluzione corrente con il miglior candidato
    current_state = best_candidate

    print(fitness(current_state))



# Iterated local search

In [114]:
NUM_ITERATIONS = 10

In [115]:
def new_starting_position(global_, last):
    if global_ is None:
        # Genera una nuova soluzione casuale se la soluzione globale è nulla
        return [choice([True, False]) for _ in range(NUM_SETS)]
    else:
        # Restituisce la soluzione globale corrente se non è nulla
        return global_

In [ ]:
global_solution = None
    
for iteration in range(NUM_ITERATIONS):
    # Fase di miglioramento locale
    current_state = new_starting_position(global_solution, None)
    current_state = local_search(current_state)
    
    # Fase di perturbazione
    new_state = tweak(current_state)
    
    # Seconda fase di miglioramento locale sulla soluzione perturbata
    new_state = local_search(new_state)
    
    # Accettazione della soluzione perturbata in base alla funzione obiettivo
    if fitness(new_state) < fitness(current_state):
        current_state = new_state
    
    # Aggiornamento della soluzione globale se necessario
    if global_solution is None or fitness(current_state) < fitness(global_solution):
        global_solution = current_state
    print(fitness(global_solution), iteration)

print(fitness(global_solution))